# Ekstrak PDF

In [1]:
import os
import fitz  # PyMuPDF
import pandas as pd

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

# Folder tempat file PDF kamu
folder_path = "PDF"

# Simpan hasil ekstraksi dalam list
data = []

for filename in os.listdir(folder_path):
    if filename.endswith(".pdf"):
        file_path = os.path.join(folder_path, filename)
        text = extract_text_from_pdf(file_path)
        data.append({
            "filename": filename,
            "konten": text
        })

# Simpan semua ke DataFrame
df = pd.DataFrame(data)

# Contoh tampilkan 1 artikel
print(df.iloc[0]['konten'][:1000])


 
 
ISSN 2302-4283 (print) 
 
ISSN 2580-9571 (online) 
 
Online di https://jurnal.poltekkes-soepraoen.ac.id 
 
DOI: 10.47794/jkhws 
10 | Jurnal Kesehatan Hesti Wira Sakti; Volume 12 Nomor 01, April 2024 
 
 
ETIKA DALAM ASURANSI KESEHATAN: MENEMUKAN 
KESEIMBANGAN DAN KEADILAN  
Annisya Putri Salsabila1, Putri Naira Kusuma2, Riswandy Wasir3, Cahya Arbitera4  
1,2,3,4Universitas Pembangunan Nasional Veteran Jakarta  
(Korespondensi: 2110713100@mahasiswa.upnvj.ac.id)  
 
ABSTRAK   
Pendahuluan: Etika merupakan komponen penting dalam berbisnis. Dalam dunia asuransi kesehatan, 
etika diperlukan untuk menunjang keadilan dan keseimbangan bagi perusahaan dan konsumen. 
Perusahaan wajib memenuhi hak-hak konsumen secara penuh, namun masih banyak perusahaan yang 
mementingkan keuntungan semata dengan melakukan kecurangan atau fraud. Melalui penelitian ini 
diharapkan memberikan wawasan mendalam tentang masalah etika dan membantu perusahaan asuransi 
kesehatan menjalankan bisnis mereka dengan cara

In [5]:
data = [
    {
        "question": "Apa tujuan dari asuransi?",
        "context": "Asuransi bertujuan untuk memberikan perlindungan finansial terhadap risiko yang tidak terduga.",
        "answers": {"text": ["memberikan perlindungan finansial"], "answer_start": [28]}
    },
    {
        "question": "Apa yang dimaksud dengan premi?",
        "context": "Premi adalah sejumlah uang yang dibayarkan oleh pemegang polis kepada perusahaan asuransi.",
        "answers": {"text": ["sejumlah uang"], "answer_start": [10]}
    }
]

import json
with open("qa_dataset.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)


In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def preprocess(example):
    inputs = tokenizer(
        example["question"],
        example["context"],
        truncation="only_second",
        max_length=384,
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length"
    )
    
    sample_mapping = inputs.pop("overflow_to_sample_mapping")
    offset_mapping = inputs.pop("offset_mapping")
    
    answers = example["answers"]
    start_char = answers["answer_start"][0]
    end_char = start_char + len(answers["text"][0])

    input_ids = inputs["input_ids"]
    context = example["context"]
    
    start_positions = []
    end_positions = []

    for i, offsets in enumerate(offset_mapping):
        input_ids_sample = input_ids[i]
        sequence_ids = inputs.sequence_ids(i)

        # Find start of context
        context_start = sequence_ids.index(1)
        context_end = len(sequence_ids) - 1 - sequence_ids[::-1].index(1)

        # Only keep offsets that belong to the context
        offsets_context = offsets[context_start:context_end]

        start = end = 0
        for idx, (start_off, end_off) in enumerate(offsets_context):
            if start_off <= start_char < end_off:
                start = context_start + idx
            if start_off < end_char <= end_off:
                end = context_start + idx
                break
        
        start_positions.append(start)
        end_positions.append(end)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

tokenized_dataset = dataset.map(preprocess, batched=False)


Map: 100%|██████████| 2/2 [00:00<00:00, 29.93 examples/s]


In [11]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained("bert-base-uncased")

training_args = TrainingArguments(
    output_dir="./qa_model",
    per_device_train_batch_size=4,
    num_train_epochs=2,
    logging_dir="./logs"
)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
from datasets import DatasetDict

dataset_dict = DatasetDict({
    "train": tokenized_dataset
})

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_dict["train"]
)

In [14]:
from transformers import pipeline

qa_pipeline = pipeline("question-answering", model=trainer.model, tokenizer=tokenizer)


Device set to use cpu


In [16]:
context = """
Asuransi kesehatan adalah bentuk perlindungan finansial terhadap biaya pengobatan.
Premi dibayarkan secara rutin kepada penyedia asuransi untuk mendapatkan manfaat perlindungan tersebut.
"""


In [17]:
question = "Apa itu asuransi kesehatan?"


In [18]:
result = qa_pipeline({
    "context": context,
    "question": question
})

print(result)


c:\Users\T480\Documents\CAPSTONE\fixenv\Lib\site-packages\transformers\pipelines\question_answering.py:390: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(


{'score': 0.0003993232094217092, 'start': 108, 'end': 138, 'answer': 'rutin kepada penyedia asuransi'}
